In [1]:
import json

In [2]:
#!pip install docx2txt

In [3]:
#!pip install python-docx
#!pip install langdetect
#!pip install textblob
#!pip install guess_language-spirit

edits to original doc: philosophy.docx

- in word doc, go to Edit > Find > Advanced Find & Replace
- in "Find what" field, type: <[A-Za-z]
- select "Use Wildcards" and (at bottom) Format > Font > Italic
- in "Replace" field, type: ∞^&
- this should insert a "∞" before every italicized word
- & do the same for underlines

new file = philosophy emph.docx

In [2]:
import docx
from guess_language import guessLanguage
import re
import json

In [3]:
doc = docx.Document("diary.docx")
all_paras = doc.paragraphs
para_list=[]
for para in all_paras:
    text=para.text
    text=re.sub("∞","<i>",text)
    text=re.sub("¡","<u>",text)
    para_list.append(text)
para_text='\n'.join(para_list)

In [8]:
# split into sections
split_mark="¬"
para_text=re.sub("Ms-10",split_mark+"Ms-10",para_text) # insert split marker
sections0=re.split(split_mark,para_text)

sections=[]
for sec in sections0:
    if len(sec)>3: # incomplete sections?
        sec=re.sub("·","•",sec)
        #sec=re.sub("\(|\)","†",sec)
        sections.append(sec)
#sections.sort()
print(len(sections))

280


In [11]:
print(sections[1])

Ms-101,1r[1]
9.8.14.
Vorgestern bei der Assentierung genommen worden & dem 2ten Festungsartillerie-Regiment in Krakau zugeteilt. Gestern vormittag von Wien ab. Komme heute vormittag in Krakau an. Guter Stimmung. Gab mein großes Schreibebuch Trenkler zur Aufbewahrung. Werde ich jetzt arbeiten können??? Sehr gespannt auf mein kommendes Leben! Die Militärbehörden in Wien waren von einer unglaublichen Freundlichkeit. Leute die von Tausenden täglich um Rat gefragt werden gaben freundliche & ausführliche Antworten. So etwas ermutigt ungeheuer. Es erinnerte mich an englische Verhältnisse. 
The day before yesterday was accepted after the medical examination for military service and assigned to the 2nd Fortress Artillery Regiment in Kraków. Left Vienna yesterday morning. Arriving in Kraków this morning. Good mood. Gave my large notebook to Trenkler for safekeeping. Will I be able to get work done now??? Very curious about my future life! The military authorities in Vienna were incredibly friend

In [15]:
# fields and their corresponding regular expressions- 
fields=[("manuscript","Ms-.*"), #Ms-101,12r[2] et 13r[1] (1914--0902) (NB)
         ("date","\n\s*([0-9]+[.][0-9]+[.][0-9]+[.]*)")] #2.9.14.

In [25]:
# grab into json
nodes=[]
empties=[]
multiples=[]
sos=[]
sections.sort()
for section in sections:
    text=[]
    
    # metadata
    s={}
    e=[]
    m=[]
    for field in fields:
        line=re.findall(field[1],section)
        if len(line)==1:
            s[field[0]]=line[0]
        elif len(line)==0:
            s[field[0]]=""
            e.append(s)
        elif len(line)>1:
            #s[field[0]]=line
            #OR
            for i in range(len(line)):
                s[field[0]+str(i+1)]=line[i]
            m.append(s)
    if len(e)>0:
        new=(section,e)
        empties.append(new)
    if m!=[]:
        new=(section,m[-1])
        multiples.append(new)
        
    # translations
    subsections=re.split("\n",section)
    ger=[]
    ger_ind=[]
    eng=[]
    eng_ind=[]
    for subsec in subsections:
        if (guessLanguage(subsec) == 'de'):
            ger.append(subsec)
            ger_ind.append(subsections.index(subsec))
        elif (guessLanguage(subsec) == 'en'):
            eng.append(subsec)
            eng_ind.append(subsections.index(subsec))
    if eng_ind!=[]:
        s["eng"]='\n'.join(eng)
    else:
        s["eng"]=""
    if ger_ind!=[]:
        s["ger"]='\n'.join(ger)
    else:
        s["ger"]=""
    if s["manuscript"]!='':
        nodes.append(s)
        
print(len(nodes))
print(len(empties))
print(len(multiples))

<ipython-input-25-b493cf412977>:41: DeprecationWarning: call to deprecated function guessLanguage()
  if (guessLanguage(subsec) == 'de'):
<ipython-input-25-b493cf412977>:44: DeprecationWarning: call to deprecated function guessLanguage()
  elif (guessLanguage(subsec) == 'en'):


279
14
8


In [26]:
with open('stern_hz_diary.json', 'w', encoding='utf8') as f:
    json.dump(nodes, f, indent=4, ensure_ascii=False)

In [17]:
# View empties
for e in empties:
    print(e[0],'\n',e[1],'\n\n•••\n')

Ms-101,24v[1]
Fast nichts gearbeitet.
Did almost no work.
 
 [{'manuscript': 'Ms-101,24v[1]', 'date': '', 'eng': '', 'ger': 'Fast nichts gearbeitet.\nDid almost no work.\n'}] 

•••

Ms-101,40v[1] 
Höre soeben daß wir heute nacht das für gestern geplante Unternehmen ausführen werden. Davon daß wir nach Krakau kommen ist noch gar keine Rede. Also heute nacht! ——! Wir sollen mit Schnellfeuerkanonen und Maschinengewehren schießen wie ich höre, mehr um Lärm zu machen als zu treffen. Auch entnehme ich daß die Sache gefährlich werden wird. Wenn ich mit dem Scheinwerfer leuchten soll so bin ich gewiß verloren. Aber das macht nichts denn nur eines ist nötig! In einer Stunde fahren wir ab. Gott ist mit mir!
Just now hear that tonight we're going to carry out the business that was planned for yesterday. There is still no talk of us going to Krakow. So tonight! ——! We shall shoot rapid firing cannons and machine guns, more to make noise than to hit anything, I am told. I gather too that it will be

In [19]:
# View multiples
for m in multiples:
    print(m[0],'\n',m[1],'\n\n\n')

Ms-102,16v[2]
15.11.14.
Lese jetzt in Emersons Essays. Vielleicht werden sie einen guten Einfluß auf mich haben. Ziemlich gearbeitet. ——.
Reading Emerson's essays now. Maybe they will have a good influence on me. Worked fairly well.  ——.
16.11.14.
 
 {'manuscript': 'Ms-102,16v[2]', 'date1': '15.11.14.', 'date2': '16.11.14.', 'eng': "Reading Emerson's essays now. Maybe they will have a good influence on me. Worked fairly well.  ——.\n16.11.14.\n", 'ger': 'Lese jetzt in Emersons Essays. Vielleicht werden sie einen guten Einfluß auf mich haben. Ziemlich gearbeitet. ——.'} 



Ms-102,63v[2]
28.2.15.
1.3.15
Nicht gearbeitet. Keine Nachricht von David! Unentschiedener und wechselnder Stimmung.
Didn’t work. No news from David! Indecisive and unsettled mood.
 
 {'manuscript': 'Ms-102,63v[2]', 'date1': '28.2.15.', 'date2': '1.3.15', 'eng': 'Didn’t work. No news from David! Indecisive and unsettled mood.\n', 'ger': 'Nicht gearbeitet. Keine Nachricht von David! Unentschiedener und wechselnder Stimm